In [422]:
# variables
T_c = 0.00000009775
f_L1 = 1575420000
N_i = 4
CN0 = 15
B_pll = 0.5
B_fe = 26000000
T = 0.020
d = 0.3
pi = math.pi
c = 299792458
B_dll = 0.5
dt = 7
sigma_uere = 0 # can't find a value for this

In [423]:
#Values from random table in Source 31
# h_0 = 1.4*(10**(-22))
# h_m1 = 2.3*(10**(-26))
# h_m2 = 3.3*(10**(-31))


#Values from sample clocks in Source 30
h_0 = 1.8*(10**(-21))
h_m1 = 6.492*(10**(-22))
h_m2 = 1.37*(10**(-24))

pi = math.pi

e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*h_m2*(pi**2)*(dt**3)
e2 = 2*h_m1*dt + h_m2*((pi*dt)**2)
e3 = ((h_0/(2*dt))+ 2*h_m1 + (8/3)*(pi*pi)*h_m2*dt)

Q = (c**2)*np.array([[e1,e2],[e2,e3]])

Q_10x10 = np.kron(np.eye(5),Q)

In [424]:
s_dll = (B_dll/(2*CN0))*(1/(T_c*B_fe))*(1+(1/(T*CN0)))
pr_var = ((c*T_c)**2)*s_dll+ sigma_uere

s_pll = (B_pll/(CN0))*(1+(1/(2*T*CN0)))
prr_var = ((c**2)/((2*pi*f_L1 * T)**2)) * (s_pll)


R = lambda N : np.kron(np.eye(N),np.array([[pr_var,0],[0,prr_var]]))

In [425]:
R_cf = lambda num_msmts: np.kron(np.eye(sum(num_msmts)),R(1))

measure_noise = lambda num_msmts : np.linalg.cholesky(R_cf(num_msmts)) @ np.random.normal(np.array([[0 for _ in range(2*sum(num_msmts))]]).T)

In [426]:
sigma_j = 10**(-16)

In [427]:
def run_both_filters(iterations, num_msmts,C,F_full,stations,stations_ckf, x, convergence = None):
    n = len(stations)
    C_ckf = (1/n)*np.ones((n,n))
    Q_full = np.kron(np.eye(n),Q)
    #Process Noise
    procc_noise = lambda : np.linalg.cholesky(Q_full) @ np.random.normal(
                                                        np.array([[0 for _ in range(2*n)]]).T)


    truth = np.zeros((iterations,2*n,1))
    measurements = []

    raw_pred_df = np.zeros((iterations,n,2,1))
    predictions_df = np.zeros((iterations,n,2,1))
    updated_pred_df = np.zeros((iterations,n,2,1))
    errors_df = np.zeros((iterations,n,2,1))
    P_hist_df = np.zeros((iterations,n,2,2))

    raw_pred_cf = np.zeros((iterations,n,2,1))
    predictions_cf = np.zeros((iterations,n,2,1))
    updated_pred_cf = np.zeros((iterations,n,2,1))
    errors_cf = np.zeros((iterations,n,2,1))
    P_hist_cf = np.zeros((iterations,n,2,2))
    #Process Noise

    T_g = 0

    for i in range(iterations):
        #This is our new truth

        x = F_full@x + procc_noise()
        truth[i] = x

        T_g += dt
        true_biases = [x[::2][i]/c for i in range(n)]
        true_drifts = [x[1::2][i]/c for i in range(n)]

        z = (H_cf(num_msmts[i]) @ truth[i]) + measure_noise(num_msmts[i])
        gps_measurements = [z[(2*sum(num_msmts[i][:k])):(2*sum(num_msmts[i][:k+1]))] for k in range(n)]
        measurements.append(z)


        if i == 0:
            time_x_dkf,gps_x_dkf,time_ests_dkf, cov_ests_dkf = diffusion_filter_iteration(
                stations,Q,R,C,gps_measurements,None,None,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])
            
            time_x_ckf,gps_x_ckf,time_ests_ckf, cov_ests_ckf = diffusion_filter_iteration(
                stations_ckf,Q,R,C_ckf,gps_measurements,None,None,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])
        else:
            time_x_dkf,gps_x_dkf,time_ests_dkf, cov_ests_dkf = diffusion_filter_iteration(
                stations,Q,R,C,gps_measurements,time_ests_dkf,cov_ests_dkf,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])
            
            time_x_ckf,gps_x_ckf,time_ests_ckf, cov_ests_ckf = diffusion_filter_iteration(
                stations_ckf,Q,R,C_ckf,gps_measurements,time_ests_ckf,cov_ests_ckf,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])

        errors_df[i] = np.array([station.x - get_station_truth(x,station.id) for station in stations])
        raw_pred_df[i] = np.array(time_x_dkf)
        updated_pred_df[i] = np.array(gps_x_dkf)
        predictions_df[i] = np.array([stn.x for stn in stations])

        P_hist_df[i] = np.array([station.P for station in stations])

        errors_cf[i] = np.array([station.x - get_station_truth(x,station.id) for station in stations_ckf])
        raw_pred_cf[i] = np.array(time_x_ckf)
        updated_pred_cf[i] = np.array(gps_x_ckf)
        predictions_cf[i] = np.array([stn.x for stn in stations_ckf])

        P_hist_cf[i] = np.array([station.P for station in stations_ckf])
        
        
    
    return errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements
